In [1]:
import pybullet as p
import sys
sys.path.append("../install/lib")
import moveit_noros as moveit
import pybullet_data
import time
import numpy as np
import threading

In [3]:
print("== Initialize Pybullet == ")
physicsClient = p.connect(p.GUI_SERVER)
#client2 = p.connect(p.SHARED_MEMORY)
#cppClient = moveit.Bullet3Hardware(physicsClient)
p.setRealTimeSimulation(1)

== Initialize Pybullet == 


In [ ]:
p.setGravity(0,0,-10)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
planeId = p.loadURDF("plane.urdf")

p.setAdditionalSearchPath("../resources")
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
boxId = p.loadURDF("pr2.urdf", startPos, startOrientation, useFixedBase=1)

print("== Initialize moveit ==")
robot_loader = moveit.createRobotModelLoaderFromFile("../resources/pr2.urdf", "../resources/pr2_config/pr2.srdf")
kinematic_solver_loader = moveit.createKinematicsLoaderFromFile("../resources/pr2_config/kinematics.yaml")
robot_loader.loadKinematicsSolvers(kinematic_solver_loader)
robot_model = robot_loader.getModel()

bulletHardware = moveit.PybulletHardware(p, boxId)


== Initialize moveit ==
